In [1]:
from fastapi import FastAPI, Form, HTTPException, Query, Request
from typing import List
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.templating import Jinja2Templates
from tensorflow.keras.models import load_model
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from fastapi.staticfiles import StaticFiles
import keras
from pytube import YouTube, Playlist 
from pytube.exceptions import VideoUnavailable,AgeRestrictedError,VideoPrivate,RegexMatchError # type: ignore

In [2]:
# Modeli yükleme
path = "bitirme.keras"
model = keras.saving.load_model(path)
model.summary()

#Tokenizer'ı yükleme
with open("bitirme.json", "r") as f:
    tokenizer = json.load(f)

tokenizer = tokenizer_from_json(tokenizer)

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ (None, 1135, 100)      │    16,879,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_33 (LSTM)                  │ (None, 1135, 64)       │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 1135, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_34 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 12)             │           396 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 5)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,805,053 (193.81 MB)

 Trainable params: 16,935,017 (64.60 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 33,870,036 (129.20 MB)

In [3]:
sentence = "Ananı sikeyim"
sentence= sentence.replace("İ","i").replace("I","ı").lower()
sentence = tokenizer.texts_to_sequences([sentence])
sentence = pad_sequences(sentence, maxlen=1135)
prediction = model.predict(sentence)
result =  prediction.argmax()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step


In [4]:
map = {
    "OTHER" : 0,
    "PROFANITY": 1,
    "INSULT" : 2,
    "RACIST" : 3,
    "SEXIST": 4
}

dsad

In [5]:
param = "https://www.youtube.com/watch?v=qc0Onh1pKcg"

yt = YouTube(param)
stream = yt.streams.get_audio_only()
try:
    captions = yt.captions["tr"]
except KeyError as error:
    print("auto_generated",error,param)
    captions = yt.captions["a.tr"]
except:
    captions = "not_found"

sentiment_list = []
for obj in captions.json_captions["events"]:
    sentiment_obj = {}
    texts = obj.get("segs",[None])
    if texts[0]:
        sentiment = ""
        if obj.get("segs",[None])[0]["utf8"] == "\n":
            continue
        for text in texts:
            if text.get("utf8","none") == "\n":
                continue
            if "[\xa0__\xa0]" in text.get("utf8","none"):
              text["utf8"] = text["utf8"].replace("[\xa0__\xa0]","amk")
            sentiment += text.get("utf8","none") + " "

        start = obj["tStartMs"]
        end = start + obj["dDurationMs"]
        sentiment_obj["sentiment"] = sentiment
        sentiment_obj["start"] = start/1000
        sentiment_obj["end"] = end/1000
        sentiment_list.append(sentiment_obj)

auto_generated 'tr' https://www.youtube.com/watch?v=qc0Onh1pKcg


In [6]:
# sentence = "Ananı sikeyim"
# sentence= sentence.replace("İ","i").replace("I","ı").lower()
# sentence = tokenizer.texts_to_sequences([sentence])
# sentence = pad_sequences(sentence, maxlen=1135)
# prediction = model.predict(sentence)
# result =  prediction.argmax()


In [7]:
link_list = []
for obj in sentiment_list:
  link_obj = {}
  text = obj["sentiment"]
  text= text.replace("İ","i").replace("I","ı").lower()
  text = tokenizer.texts_to_sequences([text])
  text = pad_sequences(text, maxlen=1135)
  prediction = model.predict(text)
  result =  prediction.argmax()
  if result != 0:
    link_obj["text"] = obj["sentiment"]
    link_obj["result"] = result
    time_stamp = int(obj["start"])
    link_obj["link"] = f"{param}&t={time_stamp}s"
    link_list.append(link_obj)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━

In [8]:
link_list

[{'text': 'gitme  chatine  gitme  gitme  yazma  bir  şey ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=41s'},
 {'text': 'hel  gitme  chatine  Hiçbir  şey  yazma ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=55s'},
 {'text': 'AB ',
  'result': 3,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=74s'},
 {'text': 'yanıma  3te  adam ',
  'result': 1,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=84s'},
 {'text': 'Duru  btr  senin  izleyeceğin  kadar  bot ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=92s'},
 {'text': 'birisi  öldü  bak  bak ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=103s'},
 {'text': 'İngiliz  konuşup  duruyor  bir  şey  kız ',
  'result': 3,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=125s'},
 {'text': 'İngiliz  İngilizce  konuşa  ingilizdir ',
  'result': 3,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=128s'},
 

In [9]:
def get_caption(captions):
    sentiment_list = []
    for obj in captions.json_captions["events"]:
        sentiment_obj = {}
        texts = obj.get("segs",[None])
        if texts[0]:
            sentiment = ""
            if obj.get("segs",[None])[0]["utf8"] == "\n":
                continue
            for text in texts:
                if text.get("utf8","none") == "\n":
                    continue
                if "[\xa0__\xa0]" in text.get("utf8","none"):
                    text["utf8"] = text["utf8"].replace("[\xa0__\xa0]","amk")
                sentiment += text.get("utf8","none") + " "

            start = obj["tStartMs"]
            end = start + obj["dDurationMs"]
            sentiment_obj["sentiment"] = sentiment
            sentiment_obj["start"] = start/1000
            sentiment_obj["end"] = end/1000
            sentiment_list.append(sentiment_obj)
    return sentiment_list

def results(sentiment_list,link):
    link_list = []
    for obj in sentiment_list:
        link_obj = {}
        text = obj["sentiment"]
        text= text.replace("İ","i").replace("I","ı").lower()
        text = tokenizer.texts_to_sequences([text])
        text = pad_sequences(text, maxlen=1135)
        prediction = model.predict(text)
        result =  prediction.argmax()
        if result != 0:
            link_obj["text"] = obj["sentiment"]
            link_obj["result"] = result
            time_stamp = int(obj["start"])
            link_obj["link"] = f"{link}&t={time_stamp}s"
            link_list.append(link_obj)
    return link_list

In [10]:
sentiment_list = get_caption(captions)
sentiment_list_with_results = results(sentiment_list,param)
print(sentiment_list_with_results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━

In [11]:
sentiment_list_with_results

[{'text': 'gitme  chatine  gitme  gitme  yazma  bir  şey ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=41s'},
 {'text': 'hel  gitme  chatine  Hiçbir  şey  yazma ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=55s'},
 {'text': 'AB ',
  'result': 3,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=74s'},
 {'text': 'yanıma  3te  adam ',
  'result': 1,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=84s'},
 {'text': 'Duru  btr  senin  izleyeceğin  kadar  bot ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=92s'},
 {'text': 'birisi  öldü  bak  bak ',
  'result': 2,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=103s'},
 {'text': 'İngiliz  konuşup  duruyor  bir  şey  kız ',
  'result': 3,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=125s'},
 {'text': 'İngiliz  İngilizce  konuşa  ingilizdir ',
  'result': 3,
  'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=128s'},
 

In [12]:
def ai_model_predict(youtube_url: str):
    yt = YouTube(youtube_url)
    stream = yt.streams.get_audio_only()
    try:
        captions = yt.captions["tr"]
    except KeyError as error:
        print("auto_generated",error,youtube_url)
        captions = yt.captions["a.tr"]
    except:
        captions = "not_found"
        
    sentiment_list = get_caption(captions)
    sentiment_list_with_results = results(sentiment_list,youtube_url)
    return sentiment_list_with_results

In [13]:
prediction = ai_model_predict("https://www.youtube.com/watch?v=kj1YlpM4AC0")

auto_generated 'tr' https://www.youtube.com/watch?v=kj1YlpM4AC0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

In [14]:
test = {}
test["https://www.youtube.com/watch?v=kj1YlpM4AC0"]  = prediction

In [16]:
test

<function __main__.test(youtube_link)>

In [55]:
for obj in sentiment_list_with_results:
    print(obj)

{'text': 'gitme  chatine  gitme  gitme  yazma  bir  şey ', 'result': 2, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=41s'}
{'text': 'hel  gitme  chatine  Hiçbir  şey  yazma ', 'result': 2, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=55s'}
{'text': 'AB ', 'result': 3, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=74s'}
{'text': 'yanıma  3te  adam ', 'result': 1, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=84s'}
{'text': 'Duru  btr  senin  izleyeceğin  kadar  bot ', 'result': 2, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=92s'}
{'text': 'birisi  öldü  bak  bak ', 'result': 2, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=103s'}
{'text': 'İngiliz  konuşup  duruyor  bir  şey  kız ', 'result': 3, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=125s'}
{'text': 'İngiliz  İngilizce  konuşa  ingilizdir ', 'result': 3, 'link': 'https://www.youtube.com/watch?v=qc0Onh1pKcg&t=128s'}
{'text': 'İngiliz  amk ', 'result': 3, 'link': 'h

In [2]:
import requests

# FastAPI uygulamanın çalıştığı URL
api_url = "http://localhost:8000/predict/"

# Göndermek istediğin YouTube URL'si
youtube_url = "https://www.youtube.com/watch?v=Bul9_jVgp-k"

# İstek verisi
data = {
    "youtube_url": youtube_url
}

# POST isteği gönder
response = requests.post(api_url, json=data)


URL: https://www.youtube.com/watch?v=Bul9_jVgp-k


KeyError: 'prediction'

In [3]:
# Yanıtı kontrol et
if response.status_code == 200:
    result = response.json()
    print(f"URL: {result['youtube_url']}")

else:
    print(f"Failed to get a response: {response.status_code}")
    print(response.json())

URL: https://www.youtube.com/watch?v=Bul9_jVgp-k


In [4]:
result

{'youtube_url': 'https://www.youtube.com/watch?v=Bul9_jVgp-k'}

In [1]:
for i in range(390):
    if i%10==0:
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
